# Redes neuronales convolucionales

Las redes neuronales convolucionales surgen a partir de buscar modelos que puedan caracterizar una imagen de un mismo objeto sin importar su rotacion o traslación.
Estas redes a diferencia de las redes neuronales perceptrón multicapa, involucran 2 nuevos conceptos: capas de convolución y capas de pooling.
En las capas de convolución básicamente se aplica un filtro a la imágen que resalta distintos rasgos, estos dependen del kernel utilizado. Despues de las capas de convolución se suele ocupar una capa de pooling, dependiendo del tipo de pooling es la operación a realizar, pero todos consisten en obtener algo significativo de un bloque de datos.
Finalmente, se llega a las capas densas donde se encuentra la interconexión de todas las neuronas, pudiendo relacionar así cada uno de los rasgos obtenidos con su etiqueta correspondiente.

In [1]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [2]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D
from keras.layers import Dense, Flatten
from keras.losses import categorical_crossentropy

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data() #Descarga un dataset de prueba
    #este es un dataset de 60000 imágenes de 28x28 en escala de grises de los 10 dígitos
    #junto con 10000 imagenes más para la validación

In [4]:
# transforma las etiquetas en one-hot-encoded vectors
num_classes = len(np.unique(y_train)) #Obtiene el número de clases diferentes
print(y_train[0], end=' => ') #Muestra la etiqueta del primer elemento de entrenamiento
y_train = keras.utils.to_categorical(y_train, 10) #Convierte los valores a variables
    #categóricas, asignando un arreglo de 10 elementos y solo uno de ellos es igual a 1,
    #los demás son ceros. La posición donde se encuentra el 1 corresponde al valor de y
y_test = keras.utils.to_categorical(y_test, 10) #realiza lo mismo para los datos de validación
print(y_train[0]) #imprime como se ve ahora la etiqueta del primer elemento de entrenamiento

5 => [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [5]:
# reescala 0-1 y convierte los datos de entrenamiento a float32
X_train = X_train.astype(np.float32) / 255 #ya que el valor de las imagenes correponde a un
X_test = X_test.astype(np.float32) / 255 #número entero entre 0-255, los convierte a 0-1
                                        #y flotante de 32 bits.
# cambia la forma de los datos
#la dimension de X_train es (60000,28,28), despues de los cambios es (60000,28,28,1)
img_rows, img_cols = X_train.shape[1:] #el primer valor de X_train.shape corresponde al numero de
        #elementos, entonces, al saltarse el primer valor obtiene la dimensión de cada uno
        #de los elementos.
X_train = X_train.reshape(len(X_train), img_rows, img_cols, 1) #asigna una dimensión extra
X_test = X_test.reshape(len(X_test), img_rows, img_cols, 1) #asigna una dimensión extra

input_shape = (img_rows, img_cols, 1) #crea un vector con la forma de los elementos
print(input_shape) #de X_train (se usa en la creación de la primera capa de la red) y lo imprime

(28, 28, 1)


In [6]:
#En este bloque de código se establece la arquitectura de la red
#Se añaden cada una de las capas, y por cada capa se establece el tipo de capa
#en caso de las capas convolucionales se establece su tamaño, el kernel y la función de activación
#para las capas de pooling se establece el tipo y tamaño de este

lenet = Sequential() #Establece el modelo de la red el cual es secuencial

# Capa Convolucional C1
lenet.add(Conv2D(6, kernel_size=(5, 5), activation='tanh',  #el primer parametro de conv2d
                 input_shape=input_shape, padding='same', name='C1')) #corresponde al numero de
        #filtros, en este caso son 6, el kernel es de 5x5, su función de activación es la
        #tangente hiperbólica, el tamaño de entrada corresponde al de las imágenes. El padding 
        #puede ser "same" o "valid" para el caso de "same", agrega ceros en los extremos de tal 
        #manera que el tamaño de la salida es "igual" que el de entrada. Para el caso de "valid" 
        #no agrega ceros y el tamaño de salida suele ser menor al de la entrada.
        #la entrada es de 28x28 y la salida de 28x28x6

# Capa de Pooling S2
lenet.add(AveragePooling2D(pool_size=(2, 2), name='S2')) #el tipo de pooling es de promedio y
        #su tamaño es de 2x2, la dimensión pasa de 28x28x6 a 14x14x6

# Capa Convolutional C3
lenet.add(Conv2D(16, kernel_size=(5, 5), activation='tanh', name='C3')) #capa de convolucion
    #con 16 filtros, kernel de 5x5 "valid" por lo que su salida es de 10x10.
    #la dimension pasa de 14x14x6 a 10x10x16

# Capa de Pooling S4
lenet.add(AveragePooling2D(pool_size=(2, 2), name='S4')) #capa de pooling utilizando el promedio
    #de tamaño de 2x2, la dimensión pasa de 10x10x96 a 5x5x16
    #

# Capa convolucional completamente conectada C5
lenet.add(Conv2D(120, kernel_size=(5, 5), activation='tanh', name='C5')) #
    #aplica 120 filtros y como solo quedan elementos de 5x5, al usar un kernel del mismo tamaño
    #el resultado es una dimension de 1x1, entonces pasa de 5x5x16 a 120x1x1

#Capa completamente conectada FC6
lenet.add(Flatten()) #la capa de flatten aplana la dimensión, es decir, pasa de una dimension
    #mayor a una dimensión N, en este caso pasa de 1x1x120 a 120
lenet.add(Dense(84, activation='tanh', name='FC6')) #capa que conecta las 120 neuronas anteriores
    #con 84

#Capa de salida (activación softmax)
lenet.add(Dense(10, activation='softmax', name='OUTPUT')) #finalmente, conecta las 84 neuronas
    #anteriores con 10 neuronas, esto corresponde con la codificación que se le dio a las etiquetas

In [7]:
lenet.compile(loss=categorical_crossentropy, optimizer='SGD', metrics=['accuracy']) #compila el 
    #modelo, en el parámetro de loss se establece la función de coste que es lo que el modelo debe 
    #de calcular como error en el entrenamiento para despues minimmizarlo, en este caso, se usa la 
    #entropia cruzada categorica la cual se recomienda para clasificar entre 2 o más clases.
    #El optimizador se refiere a como es que va a minimizar el error, aqui usa SGD o Stochastic
    #Gradient Descent. La métrica busca evaluar la eficiencia del modelo. Aqui usa accuracy 
    #que es el resultado de las clases correctamente clasificadas entre el total de la prueba.
lenet.summary() #Muesta la descripción de la red.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
C1 (Conv2D)                  (None, 28, 28, 6)         156       
_________________________________________________________________
S2 (AveragePooling2D)        (None, 14, 14, 6)         0         
_________________________________________________________________
C3 (Conv2D)                  (None, 10, 10, 16)        2416      
_________________________________________________________________
S4 (AveragePooling2D)        (None, 5, 5, 16)          0         
_________________________________________________________________
C5 (Conv2D)                  (None, 1, 1, 120)         48120     
_________________________________________________________________
flatten (Flatten)            (None, 120)               0         
_________________________________________________________________
FC6 (Dense)                  (None, 84)                1

In [8]:
#En este bloque de código se realiza el entrenamiento de la red, este llevó bastante tiempo, casi
#10 minutos.
batch_size = 64 #corresponde al número de muestras que se introduciran al mismo tiempo a la red
epochs = 50 #este es el número de epocas de entrenamiento
history = lenet.fit(X_train, y_train, #recibe los datos de entrenamiento
                      batch_size=batch_size, #el tamaño del batch
                      epochs=epochs, #el número de epocas
                      validation_data=(X_test, #los datos de validación
                                       y_test)) #con sus respectivas etiquetas.

Epoch 1/50
938/938 [==============================] - 4s 4ms/step - loss: 0.8740 - accuracy: 0.7849 - val_loss: 0.3940 - val_accuracy: 0.8932
Epoch 2/50
938/938 [==============================] - 3s 3ms/step - loss: 0.3506 - accuracy: 0.9013 - val_loss: 0.2938 - val_accuracy: 0.9136
Epoch 3/50
938/938 [==============================] - 3s 3ms/step - loss: 0.2820 - accuracy: 0.9174 - val_loss: 0.2442 - val_accuracy: 0.9285
Epoch 4/50
938/938 [==============================] - 3s 3ms/step - loss: 0.2386 - accuracy: 0.9302 - val_loss: 0.2079 - val_accuracy: 0.9364
Epoch 5/50
938/938 [==============================] - 3s 3ms/step - loss: 0.2053 - accuracy: 0.9388 - val_loss: 0.1789 - val_accuracy: 0.9463
Epoch 6/50
938/938 [==============================] - 3s 3ms/step - loss: 0.1782 - accuracy: 0.9477 - val_loss: 0.1576 - val_accuracy: 0.9542
Epoch 7/50
938/938 [==============================] - 3s 3ms/step - loss: 0.1566 - accuracy: 0.9538 - val_loss: 0.1369 - val_accuracy: 0.9604
Epoch 